<a href="https://colab.research.google.com/github/nvnsudharsan/era5_to_prism/blob/main/era5_to_prism.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install cdsapi xarray netCDF4 rioxarray geopandas matplotlib
!pip install -q xarray zarr gcsfs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.4/205.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 4.6 MB/s eta 0:00:00


In [ ]:
import xarray as xr
ds = xr.open_zarr(
    'gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3',
    chunks=None,
    storage_options={"token": "anon"}
)
ds = ds.sel(time=slice(ds.attrs['valid_time_start'], ds.attrs['valid_time_stop']))

In [ ]:
tp = ds['total_precipitation'].sel(
    time=slice("2015-01-01", "2024-12-31"),
    latitude=slice(31.0, 29.5),  # South to north!
    longitude=slice(262.0, 263.5)  # 360-based: 360 - 98 = 262
)

In [ ]:
tp_mm = tp * 1000  # Convert to mm
tp_daily = tp_mm.resample(time='1D').sum()
tp_daily.name = "tp_mm_day"

In [ ]:
tp_daily.to_netcdf("/content/drive/MyDrive/era5_tp_daily_austin_2015_2024_zarr_gcs.nc")

In [ ]:
!pip install earthengine-api geemap --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.6/462.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.1 MB/s eta 0:00:00
  Attempting uninstall: earthengine-api
    Found existing installation: earthengine-api 1.5.18
    Uninstalling earthengine-api-1.5.18:
      Successfully uninstalled earthengine-api-1.5.18


In [ ]:
import ee
ee.Authenticate()  # Run this to get the code
ee.Initialize(project='ee-my-naveensudharsan')

In [ ]:
import geemap
from datetime import datetime

# Define time range
start = '2015-01-01'
end = '2024-12-31'

# Austin region
region = ee.Geometry.Rectangle([-98.0, 29.5, -96.5, 31.0])

# PRISM daily precipitation
collection = ee.ImageCollection('OREGONSTATE/PRISM/AN81d') \
    .filterDate(start, end) \
    .filterBounds(region) \
    .select('ppt')

# Convert to a multi-band image (each day as a band)
def format_band(img):
    date_str = img.date().format('YYYYMMdd')
    return img.rename(date_str)

prism_bands = collection.map(format_band)
prism_image = prism_bands.toBands()

# Export to Drive
task = ee.batch.Export.image.toDrive(
    image=prism_image.clip(region),
    description='PRISM_Precip_2015_2024_Austin',
    folder='era5_downscaled',  # GDrive folder
    fileNamePrefix='prism_ppt_austin_2015_2024',
    region=region,
    scale=800,
    maxPixels=1e13,
    fileFormat='GeoTIFF'
)
task.start()

In [ ]:
import time

while task.active():
    print('Exporting...', task.status()['state'])
    time.sleep(60)

Exporting... RUNNING
Exporting... RUNNING
Exporting... RUNNING
Exporting... RUNNING
Exporting... RUNNING


In [ ]:
import rioxarray as rxr
import xarray as xr

# Load PRISM high-res reference
prism = rxr.open_rasterio('/content/drive/MyDrive/era5_downscaled/prism_ppt_austin_2015_2024.tif', masked=True).squeeze()

# Load ERA5 daily file you saved earlier
era5 = xr.open_dataset('/content/drive/MyDrive/era5_tp_daily_austin_2015_2024_zarr_gcs.nc')['tp_mm_day']

# Match CRS and align to PRISM grid
era5_aligned = era5.rio.write_crs("EPSG:4326").rio.reproject_match(prism)

# Save aligned ERA5 to Drive
era5_aligned.to_netcdf('/content/drive/MyDrive/era5_downscaled/era5_tp_aligned_to_prism.nc')


In [ ]:
import pandas as pd

# Load ERA5 already aligned to PRISM grid
era5 = xr.open_dataset('/content/drive/MyDrive/era5_downscaled/era5_tp_aligned_to_prism.nc')['tp_mm_day']

# Load PRISM multiband GeoTIFF
prism = rxr.open_rasterio('/content/drive/MyDrive/era5_downscaled/prism_ppt_austin_2015_2024.tif', masked=True)

# Convert to DataArray with time dimension
dates = pd.date_range('2015-01-01', periods=prism.shape[0])
prism = prism.assign_coords(band=dates).rename(band='time')

In [ ]:
# Remove single 'band' dim and set coordinates correctly
prism = prism.squeeze(drop=True)
if 'spatial_ref' in prism.coords:
    prism = prism.drop_vars('spatial_ref')

In [ ]:
trainval = prism.sel(time=slice('2015-01-01', '2022-12-31'))
test = prism.sel(time=slice('2023-01-01', '2024-12-31'))

# Split 80/20 train/val
train_dates = trainval.time.to_index()
split_idx = int(0.8 * len(train_dates))
train = trainval.sel(time=train_dates[:split_idx])
val = trainval.sel(time=train_dates[split_idx:])

# Same for ERA5
era5_train = era5.sel(time=train.time)
era5_val = era5.sel(time=val.time)
era5_test = era5.sel(time=test.time)

In [ ]:
train.name = "tp_mm_day"
val.name = "tp_mm_day"
test.name = "tp_mm_day"

era5_train.name = "tp_mm_day"
era5_val.name = "tp_mm_day"
era5_test.name = "tp_mm_day"

In [ ]:
train.to_netcdf('/content/drive/MyDrive/era5_downscaled/prism_train.nc')
val.to_netcdf('/content/drive/MyDrive/era5_downscaled/prism_val.nc')
test.to_netcdf('/content/drive/MyDrive/era5_downscaled/prism_test.nc')

era5_train.to_netcdf('/content/drive/MyDrive/era5_downscaled/era5_train.nc')
era5_val.to_netcdf('/content/drive/MyDrive/era5_downscaled/era5_val.nc')
era5_test.to_netcdf('/content/drive/MyDrive/era5_downscaled/era5_test.nc')

In [ ]:
!pip install wandb

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nvnsudharsan (nvnsudharsan-the-university-of-texas-at-austin) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
!pip install xdownscale

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 755.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 41.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
from xdownscale import Downscaler
import wandb
import torch
import os

# All models you want to try
model_list = [
    "srcnn", "fsrcnn", "lapsr", "carnm", "falsra", "falsrb", "ssresnet",
    "carn", "oisrrk2", "mdsr", "san", "rcan", "unet", "dlgsanet", "dpmn",
    "safmn", "dpt", "distgssr", "swin"
]

# Load training and test datasets (combined train+val expected)
input_da = xr.open_mfdataset([
    '/content/drive/MyDrive/era5_downscaled/era5_train.nc',
    '/content/drive/MyDrive/era5_downscaled/era5_val.nc'
]).to_array().squeeze()

target_da = xr.open_mfdataset([
    '/content/drive/MyDrive/era5_downscaled/prism_train.nc',
    '/content/drive/MyDrive/era5_downscaled/prism_val.nc'
]).to_array().squeeze()

# Load test data separately
input_test = xr.open_dataset('/content/drive/MyDrive/era5_downscaled/era5_test.nc').to_array().squeeze()
target_test = xr.open_dataset('/content/drive/MyDrive/era5_downscaled/prism_test.nc').to_array().squeeze()

# Output folder for predictions and weights
model_dir = "/content/drive/MyDrive/era5_downscaled/model_weights"
os.makedirs(model_dir, exist_ok=True)

for model_name in model_list:
    run_name = f"{model_name}_run"
    wandb.init(project="xdownscale-austin", name=run_name, reinit=True)

    # Train model using your Downscaler class
    ds = Downscaler(
        input_da=input_da,
        target_da=target_da,
        model_name=model_name,
        patch_size=64,
        batch_size=16,
        epochs=50,
        val_split=0.2,
        test_split=0.0,  # We'll use separate test data
        device='cuda' if torch.cuda.is_available() else 'cpu',
        use_wandb=True,
        patience=10
    )

    # Save best model weights manually
    torch.save(ds.model.state_dict(), f"{model_dir}/{model_name}_best.pth")

    # Predict on test set
    pred = ds.predict(input_test)
    pred.name = "tp_mm_day_pred"
    pred.to_netcdf(f"/content/drive/MyDrive/era5_downscaled/pred_{model_name}_test.nc")

    # Log test stats
    wandb.log({
        "test_mean": float(pred.mean().values),
        "test_std": float(pred.std().values)
    })
    import numpy as np

    # Compute evaluation metrics
    true = target_test.values
    predicted = pred.values

    rmse = np.sqrt(np.mean((predicted - true)**2))
    mae = np.mean(np.abs(predicted - true))
    bias = np.mean(predicted - true)
    corr = np.corrcoef(predicted.ravel(), true.ravel())[0, 1]

    wandb.log({
        "test_rmse": rmse,
        "test_mae": mae,
        "test_bias": bias,
        "test_corr": corr
    })

    wandb.finish()

wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


ValueError: too many values to unpack (expected 2)

In [ ]:
input_test

<xarray.DataArray (time: 730, y: 209, x: 210)> Size: 128MB
array([[[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
          6.3609332e-02,  6.3609332e-02,  6.3609332e-02],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
          6.3609332e-02,  6.3609332e-02,  6.3609332e-02],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
          6.3609332e-02,  6.3609332e-02,  6.3609332e-02],
        ...,
        [ 5.7853758e-03,  5.7853758e-03,  5.7853758e-03, ...,
          1.6190670e-01,  1.6190670e-01,  1.6190670e-01],
        [ 5.7853758e-03,  5.7853758e-03,  5.7853758e-03, ...,
          1.6190670e-01,  1.6190670e-01,  1.6190670e-01],
        [ 5.7853758e-03,  5.7853758e-03,  5.7853758e-03, ...,
          1.6190670e-01,  1.6190670e-01,  1.6190670e-01]],

       [[ 1.7855093e+00,  1.7855093e+00,  1.7855093e+00, ...,
          7.2914352e+00,  7.2914352e+00,  7.2914352e+00],
        [ 1.7855093e+00,  1.7855093e+00,  1.7855093e+00, ...,
          7.2914352e+00,  7.2914352e+00,  7.2914352e+00],
        [ 1.7855093e+00,  1.7855093e+00,  1.7855093e+00, ...,
          7.2914352e+00,  7.2914352e+00,  7.2914352e+00],
...
        [-4.4703484e-05, -4.4703484e-05, -4.4703484e-05, ...,
          1.8550083e-02,  1.8550083e-02,  1.8550083e-02],
        [-4.4703484e-05, -4.4703484e-05, -4.4703484e-05, ...,
          1.8550083e-02,  1.8550083e-02,  1.8550083e-02],
        [-4.4703484e-05, -4.4703484e-05, -4.4703484e-05, ...,
          1.8550083e-02,  1.8550083e-02,  1.8550083e-02]],

       [[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
        ...,
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00]]], dtype=float32)
Coordinates:
  * time      (time) datetime64[ns] 6kB 2023-01-01 2023-01-02 ... 2024-12-30
  * x         (x) float64 2kB -98.0 -98.0 -97.99 -97.98 ... -96.52 -96.51 -96.5
  * y         (y) float64 2kB 31.0 30.99 30.98 30.98 ... 29.52 29.52 29.51 29.5
    variable  <U9 36B 'tp_mm_day'

In [ ]:
target_test

<xarray.DataArray (time: 730, y: 209, x: 210)> Size: 128MB
array([[[ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        ...,
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ]],

       [[ 0.    ,  0.    ,  0.    , ...,  6.0638,  6.0638,  6.0638],
        [ 0.    ,  0.    ,  0.    , ...,  6.0638,  6.0638,  6.0638],
        [ 0.    ,  0.    ,  0.    , ...,  6.0638,  6.0638,  6.0638],
        ...,
        [ 0.    ,  0.    ,  0.    , ...,  1.3654,  1.3654,  1.3654],
        [ 0.    ,  0.    ,  0.    , ...,  1.3654,  1.3654,  1.3654],
        [ 0.    ,  0.    ,  0.    , ...,  1.3654,  1.3654,  1.3654]],

       [[ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        ...,
...
        [ 0.4563,  0.4563,  0.4563, ...,  0.    ,  0.    ,  0.    ],
        [ 0.4563,  0.4563,  0.4563, ...,  0.    ,  0.    ,  0.    ],
        [ 0.4563,  0.4563,  0.4563, ...,  0.    ,  0.    ,  0.    ]],

       [[ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        ...,
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ]],

       [[ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        ...,
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ]]],
      dtype=float32)
Coordinates:
  * x         (x) float64 2kB -98.0 -98.0 -97.99 -97.98 ... -96.52 -96.51 -96.5
  * y         (y) float64 2kB 31.0 30.99 30.98 30.98 ... 29.52 29.52 29.51 29.5
  * time      (time) datetime64[ns] 6kB 2023-01-01 2023-01-02 ... 2024-12-30
    variable  <U9 36B 'tp_mm_day'